# A Simple Decoder

Karime Maamari

---

This exercise is concerned with the "decompilation" of some set of the following input instructions:
1. **PUSH "N":** Pushes the given integer, N, onto the stack
2. **ADD:** Pops the two top values from the stack, adds them, and pushes the result
3. **SUB:** Pops the two top values from the stack, *subtracts* them, and pushes the result
4. **MUL:** Pops the two top values from the stack, *multiplies* them, and pushes the result
5. **SWAP:** Swaps the top two numbers from the stack

As an example, consider the following set of commands:
1. PUSH 1
2. PUSH 2
3. PUSH 3
4. MUL
5. ADD

First, we push the values 1, 2, and 3 onto the stack

In [1]:
stack = ["3","2","1"]
for instruction in stack:
    print("----\n",instruction,"\n----\n")

----
 3 
----

----
 2 
----

----
 1 
----



Then MUL pops off the top two terms and pushes their product onto the stack

In [219]:
stack = ["3*2","1"]
for instruction in stack:
    print("----\n",instruction,"\n----\n")

----
 3*2 
----

----
 1 
----



And, finally, ADD pops off these two values and pushes their sum back onto the stack

In [221]:
print("-------\n","3*2+1","\n-------\n")

-------
 3*2+1 
-------



The simplifier will then simplify this expression to $7$ and output to user

Continue to the input handler (two cells below) to use the decompiler

---

### The Decompiler class

In [213]:
class Decompiler:
    
    def __init__(self):
        """
        Object initialization
        
        Parameters:
        -----------
        stack: arr
            The stack object to be manipulated
        count: int
            Used to set variable names
        alphabet: list
            Variable names starting with xyz, then moving to the top of the alphabet
        """
        
        self.stack = []
        self.count = 0
        self.alphabet = list("xyzabcdefghijklmnopqrstuvw")
        
    def push(self, element):        
        """
        Pushes element into stack
        
        Parameters:
        -----------
        element: int
            value to be placed onto the stack
        """
        
        self.stack.append(str(element))
        
    def pop(self):
        """
        Pops element from stack
        """
        
        if len(self.stack):
            return self.stack.pop()
        
    def operationHelper(self, operation):
        """
        Helper function for operations.
        All operations require the general logic:
          1. If two or more terms exist, operate between the terms
          2. If one term exists, operate between term and variable
          3. If no terms exist, operate between two variables
        So we can generalize, feeding in the operation type as input
        
        Parameters:
        -----------
        operation: string
            Type of operation ("+", "-", or "*")
        """
        
        stackLength = len(self.stack)
        
        # Normal operation between two terms
        if stackLength >= 2:
            firstTerm = self.pop()
            secondTerm = self.pop()
            expression = "("+firstTerm+"{}".format(operation)+secondTerm+")"
        
        # Only one term on stack
        elif stackLength == 1:
            firstTerm = self.pop()
            expression = "("+self.alphabet[self.count]+"{}".format(operation)+firstTerm+")"
            self.count= self.count + 1
            
        # No terms on stack
        else:
            expression = "("+self.alphabet[self.count]+"{}".format(operation)+self.alphabet[self.count+1]+")"
            self.count+=2
        
        expressionArr = list(expression)
        for index in range(len(expressionArr)):
            if expressionArr[index]=='+' and expressionArr[index+1]=='-':
                expressionArr[index]='-'
                del expressionArr[index+1]
        self.push(''.join(expressionArr))

    def add(self):
        """
        Addition operation
        """
        
        self.operationHelper("+")
            
    def sub(self):
        """
        Subtraction operation
        """
        
        self.operationHelper("-")
            
    def mul(self):
        """
        Multiplication operation
        """

        self.operationHelper("*")
            
    def swap(self):
        """
        Swap the 2 elements in the stack
        """
        
        if len(self.stack) >= 2:
            firstTerm = self.pop()
            secondTerm = self.pop()
            self.push(firstTerm)
            self.push(secondTerm)
            
    def expression(self,output=False):
        """
        Returns:
        -------
        The expression on the top of the stack
        """
        if output: print(self.stack[0])
        return self.stack[0]
    
    def combinationHelper(self, string, firstIntIndex, operation):
        """
        Combines like terms within the already "simplified" expression
        For example: 3+x+y+2 -> 5+x+y
                     3*x*y*2 -> 6*x*y
        
        Parameters:
        -----------
        string: list
            The current state of the simplified string
        firstIntIndex: int
            The position of the first integer
        operation: string
            The type of operation, either '*' for multiplication or '+-' for addition/subtraction
            
        Returns:
        --------
        string: list
            The string with all combinations taken care of
        """
        # First multiplicative int
        combinedTerms=int(string[firstIntIndex])
        # Indexer for this loop
        indexer=firstIntIndex+1

        # Loop through string and combine all multiplicative terms
        while indexer<len(string):
            deletionFlag=0
            currentIsDigit = string[indexer].isdigit()
            prevIsAddSub = (string[indexer-1]=='+' or string[indexer-1]=='-')
            prevIsMult = string[indexer-1]=='*'
            atEnd = indexer==len(string)-1
            
            # If multiplication and the current char is an int and the previous term
            # is multiplication and we're not multiplying by 1
            if operation=='*' and currentIsDigit and prevIsMult and int(string[indexer])!=1:
                # Combine terms and shrink array
                combinedTerms*=int(string[indexer])
                del string[indexer]
                del string[indexer-1]
                deletionFlag=1
            
            # If addition/subtraction and current char is an int and previous term is
            # is addition/subtraction and we're at the end of the array or the next term is
            # multiplicative
            elif operation=='+-' and currentIsDigit and prevIsAddSub and (atEnd or string[indexer+1]!='*'):
                # Combine terms and shrink array
                if string[indexer-1]=='+': combinedTerms+=int(string[indexer])
                if string[indexer-1]=='-': combinedTerms-=int(string[indexer])                        
                del string[indexer-1]
                del string[indexer-1]
                deletionFlag=1

            string[firstIntIndex]=str(combinedTerms)
            indexer+=1
            if deletionFlag: indexer-=2

        return string
    
    def parenthesesHelper(self, string, openParenIndex, operation):
        """
        Takes care of parenthesis, namely focused on correctly carrying out
        cases in which parentheses are preceded by "-" or "*"
        
        Parameters:
        -----------
        string: list
            The current state of the simplified string
        openParenIndex: int
            The position of the opening parenthesis
        operation: string
            The type of operation, either '*' for multiplication or '-' for addition/subtraction      
            
        Returns:
        --------
        string: list
            The string with parentheses taken care of
        """
        tempIndex=openParenIndex+2  
        # Until we reach the close paren
        while string[tempIndex]!=')':
            # Switch the operator sign if "-(...)"
            if operation=='-' and (string[tempIndex]=='+' or string[tempIndex]=='-'):
                if(string[tempIndex]=='+'): string[tempIndex]='-'  
                elif(string[tempIndex]=='-'): string[tempIndex]='+'                  
                tempIndex+=1
            # Multiply the term before the * with the term after the operator if "*(...)"
            elif operation=='*' and (string[tempIndex]=='+' or string[tempIndex]=='-'):
                string.insert(tempIndex+1,'*')
                string.insert(tempIndex+1,string[openParenIndex-2])
            tempIndex+=1
        
        return string

    def simplify(self):
        """
        Simplifies the expression at the top of the stack
        
        Flow:
        -----
          1. Check that more than one term exists in expression
          2. If so, iterate through, removing pairs of parentheses using parenthesesHelper()
            a. If the operation before the parentheses is '-' -> flip operations within parenthetical pair
               Ex: 1-(2-3) -> 1-2+3
            b. If the operation before the parentheses is '*' -> multiply preceding term by all inner terms
               Ex: 3*(a+b) -> 3*a+3*b
          3. Now that all parenthesis are removed, further simplify the expression
            a. Remove all multiplications by 1
               Ex: 3*a*1 -> 3*a
            b. Simplify remaining products via combinationHelper()
               Ex: 3*2*a -> 6*a
            c. Remove all additions/subtractions by 0
               Ex: 3+0+a -> 3+a
            d. Move all integers to the front of resulting products
               Ex: a*b*3 -> 3*a*b
        
        Returns:
        -------
        simplifiedStr: string
            Simplified expression
        """

        # Two line solution with external library:
        #----------------------------------------#    
        from sympy import sympify as sym
        # return sym(self.expression())


        # But lets do it without the external library:
        #--------------------------------------------#
        # Take in unsimplified input expression
        simplifiedStr = list(self.expression())        
        if len(simplifiedStr)==1: return simplifiedStr[0]
        
        # Combine double digits
        # Ex: ['(','1','2','+','1',')'] -> ['(','12','+','1',')']
        index=0
        while index<len(simplifiedStr):
            while simplifiedStr[index].isdigit() and simplifiedStr[index+1].isdigit():
                simplifiedStr[index]+=simplifiedStr[index+1]
                del simplifiedStr[index+1]
                index-=1
            index+=1

        # While there is an open paren
        while simplifiedStr.count('('):

            # Flip the simplifiedStr so that .index() finds the first open paren
            # Ex: Original simplifiedStr = "(a+(b+c))", Flipped = "))c+b(+a("
            reverseIndexParen=simplifiedStr[::-1].index('(')

            # Then the index of the open paren is just the 
            # length of the simplifiedStr-1 - the reversed index
            openParenIndex = len(simplifiedStr)-1-reverseIndexParen

            # If we are dealing with a "-(...)" case        
            if simplifiedStr[openParenIndex-1]=='-':
                simplifiedStr = self.parenthesesHelper(simplifiedStr, openParenIndex, '-')

            # If we are dealing with a "*(...)" case
            elif simplifiedStr[openParenIndex-1]=='*':
                simplifiedStr = self.parenthesesHelper(simplifiedStr, openParenIndex, '*')

            # Delete parentheses pairing
            # Delete open paren
            del simplifiedStr[openParenIndex]
            # Find first index of closing paren and delete it
            del simplifiedStr[simplifiedStr.index(')')]

        # Delete *1 cases
        index=0
        while index<len(simplifiedStr):
            deletionFlag=False
            # If current is a digit (necessary check for the next condition, otherwise error thrown)
            # and this digit is an int which is preceded by * -> delete the *1
            if simplifiedStr[index].isdigit() and int(simplifiedStr[index])==1 and simplifiedStr[index-1]=='*' :
                del simplifiedStr[index]
                del simplifiedStr[index-1]
                deletionFlag=True
            index+=1
            if deletionFlag: index-=2

        # Find first multiplicative integer (for simplification)
        multIntExists=False
        index=1
        while index<len(simplifiedStr):
            # If previous index is int and next index yields *
            # Mark as first int
            if simplifiedStr[index-1].isdigit() and (simplifiedStr[index]=='*'):
                firstIntIndex=index-1
                multIntExists=True
                break 
            index+=1

        # Combine multiplicative ints
        if multIntExists:
            simplifiedStr = self.combinationHelper(simplifiedStr, firstIntIndex, '*')

        # Find first additive/subtractive (is that a word...?) int
        intExists=0
        index=0
        while index<len(simplifiedStr)-1:
            currentIsDigit = (simplifiedStr[index].isdigit())
            nextIsOperator = (simplifiedStr[index+1]=='+' or simplifiedStr[index+1]=='-')
            previousWasOperator = (simplifiedStr[index-1]=='+' or simplifiedStr[index-1]=='-')
            atBeginning = (index==0)

            # If the current value is an int followed by an operator and it's either at the beginning
            # or preceded by an int, then that is the first int!
            if currentIsDigit and nextIsOperator and (atBeginning or previousWasOperator):
                    firstIntIndex=index
                    intExists=1
                    break
            index+=1

        # If we have an additive/subtractive(?) int
        if intExists:
            simplifiedStr = self.combinationHelper(simplifiedStr, firstIntIndex, '+-')

        # Delete +/-0 cases
        index=0
        while index<len(simplifiedStr):
            deletionFlag=0
            if simplifiedStr[index].isdigit():
                if int(simplifiedStr[index])==0:
                    atBeginning = index==0
                    atEnd = index==len(simplifiedStr)-1
                    nextIsOper = (simplifiedStr[index+1]=='-' or simplifiedStr[index+1]=='+')
                    prevIsOper = (simplifiedStr[index-1]=='-' or simplifiedStr[index-1]=='+')

                    # If "0+..."
                    if atBeginning and nextIsOper:
                        del simplifiedStr[j]
                        del simplifiedStr[j]
                        deletionFlag=1
                    # If "...+0" or "...+0+..."
                    elif (atEnd and prevIsOper) or (prevIsOper and nextIsOper):
                        del simplifiedStr[j-1]
                        del simplifiedStr[j-1]
                        deletionFlag=1
            index+=1
            if deletionFlag: index-=2

        # Purely aesthetic but make sure that ints are at the beginning of products
        # Ex: 2*x*y instead of x*y*2
        index = len(simplifiedStr)-2
        while index>0:
            currIsOper = simplifiedStr[index]=='*'
            nextIsInt = simplifiedStr[index+1].isdigit()
            prevIsNotInt = not simplifiedStr[index-1].isdigit()

            # If "var*int"
            if currIsOper and nextIsInt and prevIsNotInt:
                # Swap positions of var and int
                simplifiedStr[index-1], simplifiedStr[index+1] = simplifiedStr[index+1], simplifiedStr[index-1] 
            index-=1

        return ''.join(simplifiedStr)

---

### The input handler

In [222]:
s = Decompiler()
print("Enter operation name:\n")
print("  PUSH <N>: Push an integer, N, to the stack")
print("  ADD: Add values from the stack")
print("  SUB: Subtract values from the stack")
print("  MUL: Multiply values from the stack")
print("  SWAP: Swap values from the stack")
print("  END: Enter END when finished with commands\n")

while True:
    operation = input().split()
    try:
        if len(operation)>1 and operation[0].upper()!="PUSH":
            print("Error, please limit entries to one operation per line")
            continue
        if operation[0].upper() == "PUSH":
            valueToPush = operation[1]
            s.push(int(valueToPush))
        elif operation[0].upper() == "ADD":
            s.add()
        elif operation[0].upper() == "SUB":
            s.sub()
        elif operation[0].upper() == "MUL":
            s.mul()
        elif operation[0].upper() == "SWAP":
            s.swap()
        elif operation[0].upper() == "END":
            break      
        else:
            print("Error, please enter a valid operation:\n  PUSH <n>, ADD, SUB, MUL, or SWAP")
    except:
        print("Error, please enter a single keyword unless pushing, in which case enter the keyword \"PUSH\" followed by an integer")

if len(s.stack):
    print(s.expression(),"=", s.simplify())

Enter operation name:

  PUSH <N>: Push an integer, N, to the stack
  ADD: Add values from the stack
  SUB: Subtract values from the stack
  MUL: Multiply values from the stack
  SWAP: Swap values from the stack
  END: Enter END when finished with commands

push 1
push 2
push 3
mul
add
end
((3*2)+1) = 7


---

### Test cases

In [180]:
# Test 1: 1+2=3
s = Decompiler()
s.push(1)
s.push(2)
s.add()
s.expression(output=True)
s.simplify()

(2+1)


(3, '3')

In [181]:
# Test 2: 1+2*3=7
s = Decompiler()
s.push(1)
s.push(2)
s.push(3)
s.mul()
s.add()
s.expression(output=True)
s.simplify()

((3*2)+1)


(7, '7')

In [182]:
# Test 3: 1+2*3=7
s = Decompiler()
s.push(2)
s.push(3)
s.mul()
s.push(1)
s.add()
s.expression(output=True)
s.simplify()

(1+(3*2))


(7, '7')

In [183]:
# Test 4: 1*2*x
s = Decompiler()
s.push(2)
s.mul()
s.push(1)
s.add()
s.expression(output=True)
s.simplify()

(1+(x*2))


(2*x + 1, '1+2*x')

In [184]:
# Test 5: x-1
s = Decompiler()
s.push(1)
s.sub()
s.expression(output=True)
s.simplify()

(x-1)


(x - 1, 'x-1')

In [185]:
# Test 6: 2+1*3=5
s = Decompiler()
s.push(1)
s.push(2)
s.swap()
s.push(3)
s.mul()
s.add()
s.expression(output=True)
s.simplify()

((3*1)+2)


(5, '5')

In [186]:
# Test 7: z+(x*y)
s = Decompiler()
s.mul()
s.add()
s.expression(output=True)
s.simplify()

(z+(x*y))


(x*y + z, 'z+x*y')

In [187]:
# Test 8: No operations
s = Decompiler()
s.push(1)
s.push(2)
s.push(3)
s.expression(output=True)
s.simplify()

1


'1'

In [188]:
# Test 9: -(...)
s = Decompiler()
s.add()
s.sub()
s.expression(output=True)
s.simplify()

(z-(x+y))


(-x - y + z, 'z-x-y')

In [192]:
# Test 10: Order of multiplication terms
s = Decompiler()
s.push(2)
s.mul()
s.mul()
s.expression(output=True)
s.simplify()

(y*(x*2))


(2*x*y, '2*y*x')

In [193]:
# Test 11: Order of multiplication terms
s = Decompiler()
s.push(2)
s.mul()
s.mul()
s.push(3)
s.mul()
s.expression(output=True)
s.simplify()

(3*(y*(x*2)))


(6*x*y, '6*y*x')

In [194]:
# Test 12: Empty simplify: Returns error, but in practice, the input handler would catch this
s = Decompiler()
# s.expression(output=True)
# s.simplify()

In [196]:
# Test 13: Empty swap: Returns error, but in practice, input handler would catch this
s = Decompiler()
# s.swap()
# s.expression(output=True)
# s.simplify()